In [2]:
#Import inputs

import pandas as pd

# Specify the file path and sheet name
file_path_1 = r'input\certifications_additional_metadata_INPUT.xlsx'
sheet_name_1 = 'Certificados'  # replace with the name of the sheet you want to read
# Read the sheet into a DataFrame
df_certif_adj = pd.read_excel(file_path_1, sheet_name=sheet_name_1, engine='openpyxl')
# Display the first few rows of the DataFrame

file_path_2 = r'input\email.xlsx'
sheet_name_2 = 'Plan de recursos'  # replace with the name of the sheet you want to read
# Read the sheet into a DataFrame
df_email = pd.read_excel(file_path_2, sheet_name=sheet_name_2, engine='openpyxl')

file_path_3 = r'input\export_certifications.xlsx'
sheet_name_3 = 'Certificados'  # replace with the name of the sheet you want to read
# Read the sheet into a DataFrame
df_exp_cert = pd.read_excel(file_path_3, sheet_name=sheet_name_3, engine='openpyxl')

print(df_certif_adj.columns)
print(df_email.columns)

print(df_exp_cert.columns)

Index(['EMPLOYEE CODE', 'NAME', 'SURNAME', 'ORGANIZATIONAL UNIT',
       'MANAGEMENT UNIT', 'CERTIFICATION', 'COMMENTS', 'CERTIFYING ENTITY',
       'ISSUE DATE', 'VALIDITY DATE MODIFIED', 'NORMALIZED CERTIFICATION',
       'EMAIL'],
      dtype='object')
Index(['Profess.', 'Name of the professional', 'Email', 'OU of the resource'], dtype='object')
Index(['EMPLOYEE CODE', 'NAME', 'SURNAME', 'ORGANIZATIONAL UNIT',
       'MANAGEMENT UNIT', 'CERTIFICATION', 'COMMENTS', 'CERTIFYING ENTITY',
       'ISSUE DATE', 'VALIDITY DATE'],
      dtype='object')


In [3]:
#Join all inputs

df_certif_adj = df_certif_adj[['EMPLOYEE CODE','VALIDITY DATE MODIFIED']]
merged_df_valid = pd.merge(df_exp_cert, df_certif_adj, on='EMPLOYEE CODE', how='left')  
merged_df_valid.drop(['VALIDITY DATE'], axis=1, inplace=True)
print(merged_df_valid)

df_email = df_email[['Profess.','Email']]
df_email.rename(columns={'Profess.': 'EMPLOYEE CODE'}, inplace=True)
merged_df_valid_email = pd.merge(merged_df_valid, df_email, on='EMPLOYEE CODE', how='left')  
#print(merged_df_valid_email)


      EMPLOYEE CODE           NAME    SURNAME  \
0            461682        FABIANO   GONNELLA   
1            461682        FABIANO   GONNELLA   
2            461682        FABIANO   GONNELLA   
3            461682        FABIANO   GONNELLA   
4            461682        FABIANO   GONNELLA   
...             ...            ...        ...   
6146         583461  DAVIDE FEDELE   CONTALBO   
6147         583949     ALESSANDRO  FERNANDEZ   
6148         583949     ALESSANDRO  FERNANDEZ   
6149         580883         CHIARA    MULTARI   
6150         580883         CHIARA    MULTARI   

                     ORGANIZATIONAL UNIT                      MANAGEMENT UNIT  \
0          DBT A&L Automation & Low code  Digital Business Technologies (DBT)   
1          DBT A&L Automation & Low code  Digital Business Technologies (DBT)   
2          DBT A&L Automation & Low code  Digital Business Technologies (DBT)   
3          DBT A&L Automation & Low code  Digital Business Technologies (DBT)   
4     

In [ ]:
#Normalize certifications
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI
import os
 
# Initialize envirnonment variable OPENAI_API_KEY with your API key 
with open('api_key.txt', 'r') as file:
    # Read the first line
    os.environ['OPENAI_API_KEY'] = file.readline().strip()  

# Template for the prompt
template = """Question: {question}

Answer: """

prompt = PromptTemplate(template=template,input_variables=['question'])


# OpenAI model
model = OpenAI(model_name="gpt-3.5-turbo-16k",temperature=0)


# LLMChain
llm_chain = LLMChain(
prompt=prompt,
llm=model
)

distinct_values_certifications = merged_df_valid_email['CERTIFICATION'].unique()

cert_list_str=''
for elem in distinct_values_certifications:
    cert_list_str=cert_list_str + str(elem) + '\n'

question=""" 

Under the '***' marker, you'll find a collection of technical certifications.

Some of these certifications have different text representations, but they essentially represent the same qualifications.

First, identify certifications with multiple names.

Next, standardize the names by assigning a unique one to each.

Record the results in a CSV file using a semicolon as the delimiter, including two fields:

The original certification name
The standardized certification name

For standardization:

Keep an acronym at the start of the certification name if it indicates a distinct certification.
***
"""+cert_list_str
#print(question)


# Run the model
answer = llm_chain.run(question)


In [23]:
import csv 

# Split the input string into lines
lines = answer.strip().split('\n')

# Specify the output file path (e.g., 'output.csv')
output_file_path = 'certifications_normalized.csv'

# Write the content to a CSV file with a semicolon delimiter
with open(output_file_path, 'w', newline='') as outfile:
    csv_writer = csv.writer(outfile, delimiter=';')  # Specify the delimiter
    for line in lines:
        row = line.split(';')
        csv_writer.writerow(row)
        

In [14]:
answer

'Original name of the certification;Normalized name of the certification\nCONTENT SERVER 20.2 SYSTEM ADMINISTRATOR CERTIFICATION;CONTENT SERVER 20.2 SYSTEM ADMINISTRATOR CERTIFICATION\nLEVEL 1 CREDENTIAL;LEVEL 1 CREDENTIAL\nCONTENT SERVER 16.2.4 BUSINESS CONSULTANT CERTIFICATION;CONTENT SERVER 16.2.4 BUSINESS CONSULTANT CERTIFICATION\nCERTIFICAZIONE - OPENTEXT EXTENDED ECM FOR SAP SOLUTIONS FOUNDATION V16;OPENTEXT EXTENDED ECM FOR SAP SOLUTIONS FOUNDATION V16\nCERTIFIED SIEBEL FUNDAMENTALS FOR BUSINESS ANALYST (SIEBEL 7.7);SIEBEL FUNDAMENTALS FOR BUSINESS ANALYST\nCERTIFICACION SCRUM MASTER - PSM I;SCRUM MASTER - PSM I\nGENESYS CLOUD CERTIFIED PROFESSIONAL;GENESYS CLOUD CERTIFIED PROFESSIONAL\nGENESYS CERTIFIED SIP SERVER 8 CONSULTANT;GENESYS CERTIFIED SIP SERVER 8 CONSULTANT\nGENESYS CERTIFIED INFO MART 8 CONSULTANT;GENESYS CERTIFIED INFO MART 8 CONSULTANT\nCLOUD INTEGRATION FOR APPLICATION MODERNIZATION FOUNDATION LEVEL;CLOUD INTEGRATION FOR APPLICATION MODERNIZATION FOUNDATION LEVEL

In [68]:
import pandas as pd
from io import StringIO
 
# Create a DataFrame from the openai answer
df_normalized_certifications = pd.read_csv(StringIO(answer), sep=';') 

df_output = pd.merge(merged_df_valid_email, df_normalized_certifications, left_on='CERTIFICATION', right_on='Original name of the certification', how='left')

df_output = df_output.rename(columns={'Normalized name of the certification': 'NORMALIZED CERTIFICATION'})

df_output = df_output[['EMPLOYEE CODE','NAME','SURNAME','ORGANIZATIONAL UNIT','MANAGEMENT UNIT','CERTIFICATION',
                       'COMMENTS','CERTIFYING ENTITY','ISSUE DATE','VALIDITY DATE MODIFIED','NORMALIZED CERTIFICATION']]

df_output.to_excel('output/certifications_additional_metadata_OUTPUT_IANNINI.xlsx',index = False)